# Base de Datos
### Leer base de datos

leer la base de datos "data/Base_informada" 


In [5]:
import re
import pandas as pd 
from openpyxl import load_workbook

In [42]:
LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA",
    "RG4": "-",
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}


UNIDADES= {
        "DROPA I": "UR1"
}

PERSONAL = {    
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

SCANNER = {
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}


In [43]:
contador_global_sigipol = {}

def generar_codigo_sigipol(row):
    id_operativo = row['ID_OPERATIVO']
    if id_operativo not in contador_global_sigipol:
        contador_global_sigipol[id_operativo] = 0
    contador_global_sigipol[id_operativo] += 1
    contador = contador_global_sigipol[id_operativo]
    return id_operativo + "-(" + str(contador) + ")"

# def generar_codigo_sigipol_2(row):
#         id_operativa = row['']
#         if id_operativa in conteo_acumulado:
#             conteo_acumulado[id_operativa] += 1
#         else:
#             conteo_acumulado[id_operativa] = 1
#         id_procedimiento = f"{id_operativa}-({conteo_acumulado[id_operativa]})"
#         print ("id_procedimiento: " + id_procedimiento )
#         return id_procedimiento

def generar_uid_sigpol(row):
        ursa = str(row['URSA'])
        uosp = str(row['UOSP'])
        numero_parte = str(row['NUMERO_PARTE'])
        anio = str(row['ANIO_PARTE'])
        return ursa +"-"+ uosp + "-"+ numero_parte+"-"+anio
    
        
def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"

def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"
    
def procesar_provincia(row):
    provincia = row['PROVINCIA']
    if pd.isna(provincia):
        return ""
    return PROVINCIAS.get(provincia, provincia)
    
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return UNIDADES_MUNICIPIOS.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    if pd.isna(lugar):
        return ""
    return  LUGARES_CATALOGADOS[lugar]

def procesar_direccion(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    calle = str(row['CALLE'])
    numero = str(row['NUMERO'])
    ciudad = str(row['CIUDAD'])
    partido = str(row['CIUDAD'])
    if lugar == "FUERA DE JURISDICCION" and ciudad == "ROSARIO":
        return "-"
    elif lugar == "FUERA DE JURISDICCION":
        return calle + " " + numero + ", " +ciudad + " - " + partido
    else:
        return "-"
    
def controlar_estado (row):
    ursa = row['URSA'] 
    unidad = row['UOSP'] 
    estado = row['ESTADO_PARTE'] 
    if pd.isna(unidad)  and ursa == 'RG4' and estado == 'NO DISPONIBLE ESTADISTICA':
        return  "DISPONIBLE ESTADISTICA"
    else:
        return estado 
    

def procesar_causa_judicial(row):
    causa = row.get('CAUSAJUDICIALNUMERO', '')
    if causa is None or pd.isna(causa):
        causa = ''

    tipo = str(row.get('TIPO_CAUSA_INTERNA', '')).strip()
    causa_int = str(row.get('CAUSA_INTERNA_NUMERO', '')).strip()

    if causa in ["", "S/D", "A/S", "N/C"]:
        resultado = f"{tipo}-{causa_int}".replace("--", "-")
        return resultado

    prefijos = ["NRO", "N°", "EXPTE", "EXPEDIENTE", "EXPT", "N"]

    prefijos_regex = r'\b(' + '|'.join([re.escape(prefijo) for prefijo in prefijos]) + r')\b'
    causa = re.sub(prefijos_regex, '', causa, flags=re.IGNORECASE).strip()
    causa = causa.replace("°", "").replace(".","").replace('"',"").strip()
    causa_str = re.sub(r'([A-Za-z]+)\s*(\d+)', r'\1-\2', causa)
    causa_str = causa_str.replace("--", "-").replace("---", "-")
    
    return causa_str


        
def leer_excel_a_df(worksheet):
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]
    
    data = []
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)
    
    df = pd.DataFrame(data, columns=titulos)
    return df

In [44]:
# Cargar el archivo Excel y la hoja requerida
nombre_base_2019 = "data/BASE_2017.xls"
nombre_base_2019 = "data/BASE_2018.xls"
nombre_base_2019 = "data/BASE_2019.xls"
nombre_base_2020 = "data/BASE_2020.xls"
nombre_base_2021 = "data/BASE_2021.xls"
nombre_base_2022 = "data/BASE_2022.xls"
nombre_base_2023 = "data/BASE_2023.xls"
nombre_base_2024 = "data/BASE_2024.xls"

# Leer los archivos Excel y obtener los DataFrames
base_2019 = pd.read_excel(nombre_base_2019)
base_2020 = pd.read_excel(nombre_base_2020)
base_2021 = pd.read_excel(nombre_base_2021)
base_2022 = pd.read_excel(nombre_base_2022)
base_2023 = pd.read_excel(nombre_base_2023)
base_2024 = pd.read_excel(nombre_base_2024)

# Obtener los títulos de la primera hoja
titulos = base_2019.columns

# Concatenar los DataFrames, ignorando los títulos de las demás hojas
compilado = pd.concat([base_2019, base_2020.iloc[1:], base_2021.iloc[1:], base_2022.iloc[1:], base_2023.iloc[1:], base_2024.iloc[1:]], ignore_index=True)

# Asignar los títulos a la concatenación
compilado.columns = titulos



In [48]:
df_base_de_datos =pd.DataFrame()
df_base_de_datos['UID'] = compilado.apply(generar_uid_sigpol, axis=1)
df_base_de_datos['ID_OPERATIVO'] = compilado.apply(procesar_causa_judicial, axis=1)
df_base_de_datos['FUERZA_INTERVINIENTE'] = "PSA"
df_base_de_datos['ID_PROCEDIMIENTO'] = df_base_de_datos['UID']
df_base_de_datos['UNIDAD_INTERVINIENTE'] = compilado['UOSP']
df_base_de_datos['DESCRIPCIÓN'] = compilado.apply(procesar_descripcion, axis=1)
df_base_de_datos['TIPO_INTERVENCION'] = compilado.apply(procesar_tipo, axis=1)
df_base_de_datos['PROVINCIA'] = compilado.apply(procesar_provincia, axis=1)
df_base_de_datos['DEPARTAMENTO O PARTIDO'] = compilado.apply(procesar_municipio, axis=1)
df_base_de_datos['LOCALIDAD'] = "-"
df_base_de_datos['DIRECCION'] = compilado.apply(procesar_direccion, axis=1)
df_base_de_datos['LATITUD'] = compilado['GEOREFERENCIA_Y']
df_base_de_datos['LONGITUD'] = compilado['GEOREFERENCIA_X']
df_base_de_datos['FECHA'] = pd.to_datetime(compilado['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_base_de_datos['HORA'] = pd.to_datetime(compilado['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_base_de_datos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_base_de_datos['PASO_FRONTERIZO'] = "-"
df_base_de_datos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_base_de_datos['Observaciones - Detalles'] = "-"

df_base_de_datos = df_base_de_datos[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]



* Filtro cargo lo ya informado de operaciones y lo que se informo lo concateno  a lo de operaciones de 2024

In [5]:
libro_procesado = load_workbook("data/BASE_PROCESADO.xlsx").active
base_operaciones = leer_excel_a_df(libro_procesado)

if 'ID_OPERATIVO' in base_operaciones.columns:
    base_operaciones_filtrado = base_operaciones[base_operaciones['ID_OPERATIVO'].str.contains('OSL|OSR|OSC', na=False)]
else:
    print("La columna 'ID_OPERATIVO' no existe en el DataFrame.")

df_base_de_datos = pd.concat([df_base_de_datos, base_operaciones_filtrado], ignore_index=True)


* cargo lo informado de operaciones de 2023 lo proceso y lo concate a lo que esta ya informado 

In [ ]:
libro_procesado = load_workbook("data/PLANILLA PSA CARGA DE DATOS - 2023 (OPERACIONES POLICIALES).xlsx").active
base_operaciones = leer_excel_a_df(libro_procesado)




In [ ]:
df_base_de_datos.to_excel("data/Base_2019_2024.xlsx", index=False)